In [1]:
import re
import string
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

'''
Import below doesn't needed for basic solution
It is just for experimentations
'''
import nltk
from nltk.corpus import stopwords
# Remove below comments to add necessary libraries
# import sys
# !{sys.executable} -m pip install --user -U pandas nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /home/oak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/oak/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
df = pd.read_csv('Tweets.csv', sep=',')

In [68]:
df_top_20 = pd.DataFrame()

In [73]:
df.head(20)

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,date,hashtags,mentions,sentiments
0,1160091065472487424,2019-08-10 13:00:38,False,False,False,[],1.0,0.0,0.0,# KashmirWithModi \nWith the scrapping of Art...,/Hilladventure1/status/1160091065472487424,1.154411e+18,Hilladventure1,2019-08-10,[],"[@aajtak, @RepMarkMeadows, @TheNationalUAE, @C...",0.5859
1,1160065502200205312,2019-08-10 11:19:03,False,False,False,[],0.0,0.0,0.0,Shocking & Shameful! A must read article for...,/samwhitefyr/status/1160065502200205312,9.913207e+17,samwhitefyr,2019-08-10,[],[],-0.7500
2,1161366914007875584,2019-08-14 01:30:24,False,True,False,[],0.0,0.0,0.0,"What makes "" True Islam followers"" like you co...",/rashidakhan1977/status/1161366914007875584,7.105491e+17,rashidakhan1977,2019-08-14,[],[],0.6860
3,1162074264301785088,2019-08-16 00:21:09,False,True,False,[],0.0,0.0,0.0,Just sickening to read this thread. I guess it...,/indianbyheart76/status/1162074264301785088,5.796150e+08,indianbyheart76,2019-08-16,[],[],-0.4895
4,1160566972569939969,2019-08-11 20:31:43,False,True,False,[],0.0,0.0,0.0,"Don't worry Pyari Behna, whoever opposing or p...",/ShivaprakashYe2/status/1160566972569939969,1.134819e+18,ShivaprakashYe2,2019-08-11,"[#Article, #AADHAARCARD]",[],0.2514
5,1160552349691187202,2019-08-11 19:33:36,False,False,False,[],0.0,0.0,0.0,"The Kasmiris didn't enjoy, but indulged in the...",/RishiBhargava17/status/1160552349691187202,8.054356e+17,RishiBhargava17,2019-08-11,[],[],-0.5426
6,1161871479731920896,2019-08-15 10:55:21,False,False,False,[],0.0,0.0,0.0,# KashmirWithModi \n Article 370 and 35A ga...,/abhinavchand16/status/1161871479731920896,1.161869e+18,abhinavchand16,2019-08-15,[],[],-0.5574
7,1160379841922793472,2019-08-11 08:08:07,False,True,False,[],0.0,0.0,0.0,"When a big tree falls, the earth shakes. Which...",/sshonti/status/1160379841922793472,8.010942e+17,sshonti,2019-08-11,[],[],0.0000
8,1160423711629041665,2019-08-11 11:02:27,False,True,False,[],0.0,0.0,0.0,Indian action regarding abrogation of article...,/abdulrehman670/status/1160423711629041665,2.911998e+09,abdulrehman670,2019-08-11,[],[],0.6369
9,1162297162006487041,2019-08-16 15:06:52,False,True,False,[],1.0,0.0,0.0,Well Article 370 that Hasan mentions was s...,/rahulmanuwas/status/1162297162006487041,2.972800e+08,rahulmanuwas,2019-08-16,[],[],-0.2449


In [11]:
len(pd.unique(df['usernameTweet']))

114452

In [12]:
df['date'] = df['datetime'].apply(lambda x: x.split(' ')[0])
df['medias'] = df['medias'].fillna('[]')
df['has_media'] = df['has_media'].fillna(False)
df['usernameTweet'] = df['usernameTweet'].fillna('')

In [13]:
df.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,date
0,1160091065472487424,2019-08-10 13:00:38,False,False,False,[],1.0,0.0,0.0,# KashmirWithModi \nWith the scrapping of Art...,/Hilladventure1/status/1160091065472487424,1.154411e+18,Hilladventure1,2019-08-10
1,1160065502200205312,2019-08-10 11:19:03,False,False,False,[],0.0,0.0,0.0,Shocking & Shameful! A must read article for...,/samwhitefyr/status/1160065502200205312,9.913207e+17,samwhitefyr,2019-08-10
2,1161366914007875584,2019-08-14 01:30:24,False,True,False,[],0.0,0.0,0.0,"What makes "" True Islam followers"" like you co...",/rashidakhan1977/status/1161366914007875584,7.105491e+17,rashidakhan1977,2019-08-14
3,1162074264301785088,2019-08-16 00:21:09,False,True,False,[],0.0,0.0,0.0,Just sickening to read this thread. I guess it...,/indianbyheart76/status/1162074264301785088,5.796150e+08,indianbyheart76,2019-08-16
4,1160566972569939969,2019-08-11 20:31:43,False,True,False,[],0.0,0.0,0.0,"Don't worry Pyari Behna, whoever opposing or p...",/ShivaprakashYe2/status/1160566972569939969,1.134819e+18,ShivaprakashYe2,2019-08-11


In [14]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,ID,has_media,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,user_id
ID,1,0.129047,0.189611,nan,-0.0103789,-0.00164237,-0.00498257,0.0252975
has_media,0.129047,1,-0.196213,nan,-0.00755535,0.000829262,-0.00499597,-0.0674544
is_reply,0.189611,-0.196213,1,nan,-0.025897,-0.0159754,-0.0282988,0.0494892
is_retweet,nan,nan,nan,nan,nan,nan,nan,nan
nbr_favorite,-0.0103789,-0.00755535,-0.025897,nan,1,0.433686,0.905589,-0.0259227
nbr_reply,-0.00164237,0.000829262,-0.0159754,nan,0.433686,1,0.491736,-0.0188011
nbr_retweet,-0.00498257,-0.00499597,-0.0282988,nan,0.905589,0.491736,1,-0.0260789
user_id,0.0252975,-0.0674544,0.0494892,nan,-0.0259227,-0.0188011,-0.0260789,1


In [54]:
#df_top_20['medias'] = df['medias'].value_counts().values[1:21]

array([93, 67, 50, 25, 24, 23, 22, 18, 16, 15, 15, 13, 13, 13, 13, 12, 11,
       10, 10, 10])

In [22]:
df['hashtags'] = df['text'].apply( lambda x: re.findall(r'#\S+', x))
df['mentions'] = df['text'].apply( lambda x: re.findall(r'@\S+', x))

In [23]:
df.head(100)

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,date,hashtags,mentions
0,1160091065472487424,2019-08-10 13:00:38,False,False,False,[],1.0,0.0,0.0,# KashmirWithModi \nWith the scrapping of Art...,/Hilladventure1/status/1160091065472487424,1.154411e+18,Hilladventure1,2019-08-10,[],"[@aajtak, @RepMarkMeadows, @TheNationalUAE, @C..."
1,1160065502200205312,2019-08-10 11:19:03,False,False,False,[],0.0,0.0,0.0,Shocking & Shameful! A must read article for...,/samwhitefyr/status/1160065502200205312,9.913207e+17,samwhitefyr,2019-08-10,[],[]
2,1161366914007875584,2019-08-14 01:30:24,False,True,False,[],0.0,0.0,0.0,"What makes "" True Islam followers"" like you co...",/rashidakhan1977/status/1161366914007875584,7.105491e+17,rashidakhan1977,2019-08-14,[],[]
3,1162074264301785088,2019-08-16 00:21:09,False,True,False,[],0.0,0.0,0.0,Just sickening to read this thread. I guess it...,/indianbyheart76/status/1162074264301785088,5.796150e+08,indianbyheart76,2019-08-16,[],[]
4,1160566972569939969,2019-08-11 20:31:43,False,True,False,[],0.0,0.0,0.0,"Don't worry Pyari Behna, whoever opposing or p...",/ShivaprakashYe2/status/1160566972569939969,1.134819e+18,ShivaprakashYe2,2019-08-11,"[#Article, #AADHAARCARD]",[]
5,1160552349691187202,2019-08-11 19:33:36,False,False,False,[],0.0,0.0,0.0,"The Kasmiris didn't enjoy, but indulged in the...",/RishiBhargava17/status/1160552349691187202,8.054356e+17,RishiBhargava17,2019-08-11,[],[]
6,1161871479731920896,2019-08-15 10:55:21,False,False,False,[],0.0,0.0,0.0,# KashmirWithModi \n Article 370 and 35A ga...,/abhinavchand16/status/1161871479731920896,1.161869e+18,abhinavchand16,2019-08-15,[],[]
7,1160379841922793472,2019-08-11 08:08:07,False,True,False,[],0.0,0.0,0.0,"When a big tree falls, the earth shakes. Which...",/sshonti/status/1160379841922793472,8.010942e+17,sshonti,2019-08-11,[],[]
8,1160423711629041665,2019-08-11 11:02:27,False,True,False,[],0.0,0.0,0.0,Indian action regarding abrogation of article...,/abdulrehman670/status/1160423711629041665,2.911998e+09,abdulrehman670,2019-08-11,[],[]
9,1162297162006487041,2019-08-16 15:06:52,False,True,False,[],1.0,0.0,0.0,Well Article 370 that Hasan mentions was s...,/rahulmanuwas/status/1162297162006487041,2.972800e+08,rahulmanuwas,2019-08-16,[],[]


In [24]:
df.loc[(df['hashtags'].str.len() != 0),:].count()

ID               123238
datetime         123238
has_media        123238
is_reply         123238
is_retweet       123238
medias           123238
nbr_favorite     123238
nbr_reply        123238
nbr_retweet      123238
text             123238
url              123238
user_id          123238
usernameTweet    123238
date             123238
hashtags         123238
mentions         123238
dtype: int64

In [25]:
sid = SentimentIntensityAnalyzer()
df['sentiments'] = df['text'].apply(lambda x: sid.polarity_scores(x).get('compound'))

In [76]:
pd.DataFrame(df['medias'].values.tolist()).stack().value_counts().head(21)

['[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['https://t.co/V2OJuyBmTi']",
 "['https://t.co/pWp7Nq1eEX']",
 '[]',
 '[]',
 '[]',
 "['https://t.co/V2CZN6V1PI']",
 "['https://t.co/fUgJptnBHj']",
 '[]',
 '[]',
 "['https://t.co/UeYFXThSZm']",
 '[]',
 '[]',
 '[]',
 "['https://t.co/QMvAcdCbcD']",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['https://t.co/bbcT8SvvtG']",
 '[]',
 '[]',
 '[]',
 '[]',
 "['https://t.co/F6HfWrYQBp']",
 '[]',
 '[]',
 "['https://t.co/mVipqiaL2Z']",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['https://t.co/RwuRKKTDaV']",
 "['https://t.co/HHUsS9qrPy']",
 '[]',
 "['https://t.co/fsOFwDMWck']",
 '[]',
 '[]',
 '[]',
 "['https://t.co/Za0XEdcB0V']",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['https://t.co/UDc6fbHnJz']",
 '[]',
 "['https://t.co/9LHpMhK1sx']",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['https://t.co/uWg9SVdjFE']",
 "['https://t.co/rXkcdUpWKa']",
 '[]',
 '[]',
 '[]',
 '[]',
 

In [88]:
df_top_20 = pd.DataFrame()
df_top_20['medias'] = pd.DataFrame(df['medias'].values.tolist()).stack().value_counts().head(21)
fix_medias = []
for x in df_top_20.iterrows():
    fix_item = x[0].replace('[', '')
    fix_item = fix_item.replace(']', '')
    fix_medias.append(fix_item)

df_top_20.index = fix_medias
df_top_20.to_csv('top_20_medias.csv')

In [89]:
df_top_20 = pd.DataFrame()
df_top_20['hashtags'] = pd.DataFrame(df['hashtags'].values.tolist()).stack().value_counts().head(20)
df_top_20.to_csv('top_20_hashtags.csv')

In [90]:
df_top_20 = pd.DataFrame()
df_top_20['mentions'] = pd.DataFrame(df['mentions'].values.tolist()).stack().value_counts().head(20)
df_top_20.to_csv('top_20_mentions.csv')

In [91]:
df_top_20 = pd.DataFrame()
df_top_20['usernameTweet'] = pd.DataFrame(df['usernameTweet'].values.tolist()).stack().value_counts().head(20)
df_top_20.to_csv('top_20_usernameTweet.csv')

In [67]:
df.head(10)

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,date,hashtags,mentions,sentiments
0,1160091065472487424,2019-08-10 13:00:38,False,False,False,[],1.0,0.0,0.0,# KashmirWithModi \nWith the scrapping of Art...,/Hilladventure1/status/1160091065472487424,1.154411e+18,Hilladventure1,2019-08-10,[],"[@aajtak, @RepMarkMeadows, @TheNationalUAE, @C...",0.5859
1,1160065502200205312,2019-08-10 11:19:03,False,False,False,[],0.0,0.0,0.0,Shocking & Shameful! A must read article for...,/samwhitefyr/status/1160065502200205312,9.913207e+17,samwhitefyr,2019-08-10,[],[],-0.7500
2,1161366914007875584,2019-08-14 01:30:24,False,True,False,[],0.0,0.0,0.0,"What makes "" True Islam followers"" like you co...",/rashidakhan1977/status/1161366914007875584,7.105491e+17,rashidakhan1977,2019-08-14,[],[],0.6860
3,1162074264301785088,2019-08-16 00:21:09,False,True,False,[],0.0,0.0,0.0,Just sickening to read this thread. I guess it...,/indianbyheart76/status/1162074264301785088,5.796150e+08,indianbyheart76,2019-08-16,[],[],-0.4895
4,1160566972569939969,2019-08-11 20:31:43,False,True,False,[],0.0,0.0,0.0,"Don't worry Pyari Behna, whoever opposing or p...",/ShivaprakashYe2/status/1160566972569939969,1.134819e+18,ShivaprakashYe2,2019-08-11,"[#Article, #AADHAARCARD]",[],0.2514
5,1160552349691187202,2019-08-11 19:33:36,False,False,False,[],0.0,0.0,0.0,"The Kasmiris didn't enjoy, but indulged in the...",/RishiBhargava17/status/1160552349691187202,8.054356e+17,RishiBhargava17,2019-08-11,[],[],-0.5426
6,1161871479731920896,2019-08-15 10:55:21,False,False,False,[],0.0,0.0,0.0,# KashmirWithModi \n Article 370 and 35A ga...,/abhinavchand16/status/1161871479731920896,1.161869e+18,abhinavchand16,2019-08-15,[],[],-0.5574
7,1160379841922793472,2019-08-11 08:08:07,False,True,False,[],0.0,0.0,0.0,"When a big tree falls, the earth shakes. Which...",/sshonti/status/1160379841922793472,8.010942e+17,sshonti,2019-08-11,[],[],0.0000
8,1160423711629041665,2019-08-11 11:02:27,False,True,False,[],0.0,0.0,0.0,Indian action regarding abrogation of article...,/abdulrehman670/status/1160423711629041665,2.911998e+09,abdulrehman670,2019-08-11,[],[],0.6369
9,1162297162006487041,2019-08-16 15:06:52,False,True,False,[],1.0,0.0,0.0,Well Article 370 that Hasan mentions was s...,/rahulmanuwas/status/1162297162006487041,2.972800e+08,rahulmanuwas,2019-08-16,[],[],-0.2449


In [92]:
df_copy = df[['nbr_favorite', 'nbr_reply', 'nbr_retweet', 'usernameTweet', 'date', 'sentiments']]

In [93]:
df_copy.to_csv('tweet_datasource.csv', sep=',', index=False)